In [63]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

In [64]:
torch.cuda.is_available()

True

In [61]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   2549 MiB |   4415 MiB |  39025 GiB |  39023 GiB |\n|       from large pool |   2546 MiB |   4412 MiB |  39019 GiB |  39017 GiB |\n|       from small pool |      2 MiB |      3 MiB |      5 GiB |      5 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   2549 MiB |   4415 MiB |  39025 GiB |  39023 GiB |\n|       from large pool |   2546 MiB |   4412 MiB |

In [62]:
torch.cuda.empty_cache()

In [40]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [41]:
device=torch.device('cuda:0')

In [42]:

# Load a pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
content = pd.read_csv('content.csv', sep=',')
label = pd.read_csv('label.csv', sep=',')
message_id = pd.read_csv('message_id.csv', sep=',')
content=content.values.tolist()
label=label.values.tolist()
message_id=message_id.values.tolist()

In [44]:
content=[str(item) for sublist in content for item in sublist]
label=[item-1 for sublist in label for item in sublist]
message_id=[item for sublist in message_id for item in sublist]

In [45]:

# tokenize 
inputs = tokenizer(content, padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(label)

# batching batch descent
dataset = TensorDataset(inputs.input_ids, inputs.attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=5, sampler=RandomSampler(dataset))


optimizer = AdamW(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# to gpu
model=model.to(device)


# Training loop
num_epochs=2
for epoch in range(num_epochs):
    print(epoch)
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        
        input_ids=input_ids.to(device)
        attention_mask=attention_mask.to(device)
        labels=labels.to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation and inference steps would follow here

C:\Users\hp\miniconda3\envs\tf_gpu\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0
1


In [57]:
# Replace 'new_text' with the text you want to classify
new_text = "I hope this email finds you well. My name is John Smith, and I am a student in your Intro to European History class.  "

# Tokenize and preprocess the input text
input_data = tokenizer(new_text, padding=True, truncation=True, return_tensors="pt")
input_ids = input_data.input_ids.to(device)
attention_mask = input_data.attention_mask.to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Get the predicted class probabilities or logits
logits = outputs.logits

# If you want class probabilities, you can apply a softmax function
probs = torch.softmax(logits, dim=1)

# If you want the predicted class labels, you can find the argmax
predicted_labels = torch.argmax(logits, dim=1)

In [58]:
probs

tensor([[5.2345e-02, 2.5044e-04, 4.1016e-02, 8.7724e-01, 2.3429e-02, 4.3960e-03,
         9.5913e-04, 3.6388e-04]], device='cuda:0')